In [1]:
using Distributed
addprocs(8);

In [2]:
@everywhere using PulseInputDDM, Flatten

### Generate some data

Create an instance of the `θchoice` [composite type](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) which contains the 9 parameters of a choice DDM model. 

In [3]:
θ_syn = θchoice(θz=θz(σ2_i = 1., B = 13., λ = -0.5, σ2_a = 10., σ2_s = 1.0,
    ϕ = 0.4, τ_ϕ = 0.02), bias=0.1, lapse=0.1);

In [4]:
xgen = collect(Flatten.flatten(θ_syn));

Generate 20K trials of the synthetic data using those parameters. change `rng` to get a different set with the same parameters. `dt` specifies the temporal binning of the data. `1e-2` has worked well.

In [5]:
_, data = synthetic_data(;θ=θ_syn, ntrials=8_000, rng=2, dt=1e-2);

In [6]:
model = choiceDDM(θ=θ_syn)


choiceDDM{θchoice{θz{Float64}, Float64}}
  θ: θchoice{θz{Float64}, Float64}
  n: Int64 53
  cross: Bool false


In [7]:
options = choiceoptions(lb=vcat([0., 2.,  -5., 0.,   0.,  0., 0.005, -5.0, 0.0]),
    ub = [30., 100., 5., 200., 10., 1.2,  1., 5.0, 1.0], 
    fit = trues(9))

choiceoptions
  fit: Array{Bool}((9,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((9,)) [0.0, 2.0, -5.0, 0.0, 0.0, 0.0, 0.005, -5.0, 0.0]
  ub: Array{Float64}((9,)) [30.0, 100.0, 5.0, 200.0, 10.0, 1.2, 1.0, 5.0, 1.0]


## Saving and reloading modeling results and saving data

Use the `save_choice_model` function to save the optimized model, the options used to define the optimization, and the confidence intervals

In [8]:
save_file = "./data/example_results.mat"
save_choice_model(save_file, model, options)

if you want to restart the optimization from where you stopped `reload_choice_model` will reload those model parameters and the `options`

In [9]:
θ, options = reload_choice_model(save_file);

Save the choice data using `save_choice_data` and then reload it using `load_choice_data`

In [10]:
import PulseInputDDM: save_choice_data

In [11]:
save_choice_data("./data/example_matfile_2.mat", data)

In [12]:
data = load_choice_data("./data/example_matfile_2.mat");

## Loading data and fitting a choice model from real data

Because many neuroscientists use matlab, we use the [MAT.jl](https://github.com/JuliaIO/MAT.jl) package for IO. Data can be loaded using two conventions. One of these conventions is easier when data is saved within matlab as a .MAT file, and is described below. 

The package expects your data to live in a single .mat file which should contain a struct called `rawdata`. Each element of `rawdata` should have data for one behavioral trial and `rawdata` should contain the following fields with the specified structure:

 - `rawdata.leftbups`: row-vector containing the relative timing, in seconds, of left clicks on an individual trial. 0 seconds is the start of the click stimulus.
 - `rawdata.rightbups`: row-vector containing the relative timing in seconds (origin at 0 sec) of right clicks on an individual trial. 0 seconds is the start of the click stimulus.
 - `rawdata.T`: the duration of the trial, in seconds. The beginning of a trial is defined as the start of the click stimulus. The end of a trial is defined based on the behavioral event “cpoke_end”. This was the Hanks convention.
 - `rawdata.pokedR`: `Bool` representing the animal choice (1 = right).
 
The example file located at `../choice model/example_matfile.mat` adheres to this convention and can be loaded using the `load_choice_data` method.

In [13]:
data = load_choice_data("./data/example_matfile.mat");